In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoTokenizer
import kagglehub
import wandb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotion-classifier/pytorch/v1/3/emotion_classifier.pth
/kaggle/input/emotion-classifier/pytorch/v1/3/best_model.pt
/kaggle/input/emotion-classifier/pytorch/v1/1/emotion_classifier.pth
/kaggle/input/debertafull/pytorch/v1/1/spm.model
/kaggle/input/debertafull/pytorch/v1/1/config.json
/kaggle/input/debertafull/pytorch/v1/1/training_args.bin
/kaggle/input/debertafull/pytorch/v1/1/tokenizer.json
/kaggle/input/debertafull/pytorch/v1/1/tokenizer_config.json
/kaggle/input/debertafull/pytorch/v1/1/model.safetensors
/kaggle/input/debertafull/pytorch/v1/1/special_tokens_map.json
/kaggle/input/debertafull/pytorch/v1/1/added_tokens.json
/kaggle/input/roberta-emotion/pytorch/v1/1/model.pth
/kaggle/input/2025-sep-dl-gen-ai-project/sample_submission.csv
/kaggle/input/2025-sep-dl-gen-ai-project/train.csv
/kaggle/input/2025-sep-dl-gen-ai-project/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/train.csv')
test = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/test.csv')
sample = pd.read_csv("/kaggle/input/2025-sep-dl-gen-ai-project/sample_submission.csv")

In [3]:
train.head(5)

,id,text,anger,fear,joy,sadness,surprise,emotions
0,0,the dentist that did the work apparently did a...,1,0,0,1,0,['anger' 'sadness']
1,1,i'm gonna absolutely ~~suck~~ be terrible duri...,0,1,0,1,0,['fear' 'sadness']
2,2,"bridge: so leave me drowning calling houston, ...",0,1,0,1,0,['fear' 'sadness']
3,3,after that mess i went to see my now ex-girlfr...,1,1,0,1,0,['anger' 'fear' 'sadness']
4,4,"as he stumbled i ran off, afraid it might some...",0,1,0,0,0,['fear']


In [6]:
test.head(5)

,id,text
0,0,she wanted to fight over every single little t...
1,1,"anyway, back to tuesday."
2,2,she shrieked at the dog to go back.
3,3,yelling for everyone to get back or get inside...
4,4,still kind of freaky.


In [5]:
print(train.shape)
print(test.shape)

(6827, 8)
(1707, 2)


# **EDA** 

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
x = train.iloc[:,2:7].sum()
rowsums = train.iloc[:, 2:7].sum(axis=1) # take label columns and sum it column wise
no_label_count = 0 
for i, count in rowsums.items():     
	if count==0:         
		no_label_count += 1          
print('Total number of rows:', len(train)) 
print('Total number of rows without labels(emotions):', no_label_count) 	
print('Total labels:', x.sum())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(6, 4))
ax = sns.barplot(x=x.index, y=x.values, alpha=0.8, palette=['tab:blue', 'tab:orange', 'tab:green', 'tab:brown', 'tab:red', 'tab:grey'])
plt.title('Label Counts')
plt.ylabel('Count')
plt.xlabel('Emotion')

plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
ax = sns.countplot(x=rowsums.values, alpha=0.8, palette=['tab:blue', 'tab:orange', 'tab:green', 'tab:brown', 'tab:red', 'tab:grey'])
plt.title('Emotions per Row')
plt.ylabel('# of Occurences')
plt.xlabel('# of Labels')
plt.show()

In [ ]:
train.iloc[6750,1]

# **DEBERTA V3 BASE(0.804)**

In [5]:
# # Imports & seed setup
# import os
# import random
# import numpy as np
# import pandas as pd
# import torch
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score
# from datasets import Dataset
# from transformers import (
#     AutoTokenizer,
#     AutoModelForSequenceClassification,
#     DataCollatorWithPadding,
#     TrainingArguments,
#     Trainer,
#     set_seed
# )


# # Reproducibility
# SEED = 42
# set_seed(SEED)
# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(SEED)


2025-11-30 15:07:20.375937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764515240.566280      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764515240.625652      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# wandb.login(key = "1cae1eb0b3009c258573b649b577124df891befe" , relogin=True) 

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: somya108 (dlproject_sept2025) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [12]:
# train_df = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/train.csv')
# test_df = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/test.csv')


# label_cols = ['anger','fear','joy','sadness','surprise']
# train_df[label_cols] = train_df[label_cols].astype(int)

# train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=SEED, shuffle=True)
# train_df = train_df.reset_index(drop=True)
# val_df   = val_df.reset_index(drop=True)

# # Create HuggingFace Datasets
# hf_train = Dataset.from_pandas(train_df[['id','text'] + label_cols])
# hf_val   = Dataset.from_pandas(val_df[['id','text'] + label_cols])
# hf_test  = Dataset.from_pandas(test_df[['id','text']])

# def add_labels(example):
#     example["labels"] = [float(example[c]) for c in label_cols]  # <-- float
#     return example

# hf_train = hf_train.map(add_labels)
# hf_val   = hf_val.map(add_labels)



# MODEL_NAME = "microsoft/deberta-v3-base"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
# MAX_LENGTH = 256

# def tokenize(batch):
#     return tokenizer(batch["text"], truncation=True, padding=False, max_length=MAX_LENGTH)

# hf_train = hf_train.map(tokenize, batched=True)
# hf_val   = hf_val.map(tokenize, batched=True)
# hf_test  = hf_test.map(tokenize, batched=True)


# hf_train = hf_train.remove_columns(label_cols)
# hf_val   = hf_val.remove_columns(label_cols)


# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/6144 [00:00<?, ? examples/s]

Map:   0%|          | 0/683 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/6144 [00:00<?, ? examples/s]

Map:   0%|          | 0/683 [00:00<?, ? examples/s]

Map:   0%|          | 0/1707 [00:00<?, ? examples/s]

In [13]:
# model = AutoModelForSequenceClassification.from_pretrained(
#     MODEL_NAME,
#     num_labels=len(label_cols),
#     problem_type="multi_label_classification"
# )

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     probs = torch.sigmoid(torch.tensor(logits))
#     preds = (probs > 0.5).int().numpy()
#     labels = labels.astype(int)
#     macro_f1 = f1_score(labels, preds, average="macro", zero_division=0)
#     per_label_f1 = f1_score(labels, preds, average=None, zero_division=0).tolist()
#     return {
#         "macro_f1": macro_f1,
#         "f1_anger": per_label_f1[0],
#         "f1_fear": per_label_f1[1],
#         "f1_joy": per_label_f1[2],
#         "f1_sadness": per_label_f1[3],
#         "f1_surprise": per_label_f1[4],
#     }

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# training_args = TrainingArguments(
#     output_dir="./deberta_v3_base_emotion",

#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=16,
#     gradient_accumulation_steps=1,

#     learning_rate=2e-5,
#     weight_decay=0.01,
#     num_train_epochs=3,

#     eval_strategy="steps",
#     save_strategy="steps",
#     eval_steps=500,
#     save_steps=500,
#     logging_steps=100,

#     load_best_model_at_end=True,
#     metric_for_best_model="macro_f1",

#     save_total_limit=2,
#     fp16=True if torch.cuda.is_available() else False,
#     seed=SEED,

#     report_to="wandb",     
#     run_name="DeBERTa-v3-emotion", 
    
# )

# wandb.init(project="23f1001420-t32025", name="deberta-v3-base") 

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=hf_train,
#     eval_dataset=hf_val,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

/tmp/ipykernel_38/362676207.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
# trainer.train()
# trainer.save_model()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Macro F1,F1 Anger,F1 Fear,F1 Joy,F1 Sadness,F1 Surprise
500,0.302200,0.295517,0.752987,0.569106,0.853567,0.764706,0.780723,0.796834
1000,0.215900,0.246335,0.824202,0.733333,0.879899,0.800000,0.845794,0.861985


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [16]:
# wandb.finish()  

eval/f1_anger,▁█
eval/f1_fear,▁█
eval/f1_joy,▁█
eval/f1_sadness,▁█
eval/f1_surprise,▁█
eval/loss,█▁
eval/macro_f1,▁█
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▂▂▃▄▄▄▅▆▆▇▇██


In [10]:
# save_dir = "deberta_full_model"
# os.makedirs(save_dir, exist_ok=True)

# # Save HF model
# trainer.save_model(save_dir)

# # Save tokenizer
# tokenizer.save_pretrained(save_dir)

# print("Saved full model folder:", os.listdir(save_dir))
# user = "somya2611"
# Deberta_handle = f"{user}/DebertaFull/pyTorch/v1"

# kagglehub.model_upload(Deberta_handle, "deberta_full_model")


Saved full model folder: ['training_args.bin', 'spm.model', 'special_tokens_map.json', 'model.safetensors', 'tokenizer.json', 'added_tokens.json', 'tokenizer_config.json', 'config.json']
Uploading Model https://www.kaggle.com/models/somya2611/DebertaFull/pyTorch/v1 ...
Model 'DebertaFull' does not exist or access is forbidden for user 'somya2611'. Creating or handling Model...
Model 'DebertaFull' Created.
Starting upload for file deberta_full_model/training_args.bin


Uploading: 100%|██████████| 5.30k/5.30k [00:00<00:00, 14.1kB/s]

Upload successful: deberta_full_model/training_args.bin (5KB)
Starting upload for file deberta_full_model/spm.model


Uploading: 100%|██████████| 2.46M/2.46M [00:00<00:00, 5.41MB/s]

Upload successful: deberta_full_model/spm.model (2MB)
Starting upload for file deberta_full_model/special_tokens_map.json


...

Uploading: 100%|██████████| 1.31k/1.31k [00:00<00:00, 3.38kB/s]

Upload successful: deberta_full_model/tokenizer_config.json (1KB)
Starting upload for file deberta_full_model/config.json


Uploading: 100%|██████████| 1.13k/1.13k [00:00<00:00, 2.94kB/s]

Upload successful: deberta_full_model/config.json (1KB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/somya2611/DebertaFull/pyTorch/v1


In [12]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# user = "somya2611"
# Deberta_handle = f"{user}/DebertaFull/pyTorch/v1"

# model_folder = kagglehub.model_download(Deberta_handle)
# print("Files downloaded:", os.listdir(model_folder))


Files downloaded: ['spm.model', 'config.json', 'training_args.bin', 'tokenizer.json', 'tokenizer_config.json', 'model.safetensors', 'special_tokens_map.json', 'added_tokens.json']


In [13]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained(model_folder)

# model = AutoModelForSequenceClassification.from_pretrained(
#     model_folder,
#     problem_type="multi_label_classification"
# ).to(device)

# model.eval()


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [14]:
# inference_trainer = Trainer(
#     model=model,
#     tokenizer=tokenizer,
#     data_collator=data_collator
# )

# preds = inference_trainer.predict(hf_test)
# logits = preds.predictions
# probs = torch.sigmoid(torch.tensor(logits)).numpy()
# binary_preds = (probs > 0.5).astype(int)


# submission = pd.DataFrame(binary_preds, columns=label_cols)
# submission.insert(0, "id", test_df["id"].values)

# submission.to_csv("submission.csv", index=False)
# print("Saved submission.csv")
# submission.head()


/tmp/ipykernel_38/3727258776.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  inference_trainer = Trainer(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Saved submission.csv


,id,anger,fear,joy,sadness,surprise
0,0,1,1,0,0,0
1,1,0,0,0,0,0
2,2,1,0,0,0,0
3,3,0,1,0,0,0
4,4,0,1,0,0,1


# **ROBERTA(0.81)**

In [17]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
# from tqdm import tqdm

# train.head()


,id,text,anger,fear,joy,sadness,surprise,emotions
0,0,the dentist that did the work apparently did a...,1,0,0,1,0,['anger' 'sadness']
1,1,i'm gonna absolutely ~~suck~~ be terrible duri...,0,1,0,1,0,['fear' 'sadness']
2,2,"bridge: so leave me drowning calling houston, ...",0,1,0,1,0,['fear' 'sadness']
3,3,after that mess i went to see my now ex-girlfr...,1,1,0,1,0,['anger' 'fear' 'sadness']
4,4,"as he stumbled i ran off, afraid it might some...",0,1,0,0,0,['fear']


In [18]:
# wandb.login(key = "1cae1eb0b3009c258573b649b577124df891befe" , relogin=True) 

# wandb.init(
#     project="23f1001420-t32025",   
#     name="roberta_trainer", 
#     config={
#         "model_name": "roberta-base",
#         "batch_size": 16,
#         "lr": 2e-5,
#         "epochs": 3,
#         "max_len": 128
#     }
# )
# config = wandb.config   

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [19]:
label_cols = ["anger","fear","joy","sadness","surprise"]

X_train, X_val = train_test_split(train, test_size=0.1, random_state=42, shuffle=True)
class EmotionDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128, is_train=True):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_train = is_train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = str(row["text"])

        enc = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        if self.is_train:
            labels = torch.tensor(row[label_cols].values.astype('float32'))
            return enc.input_ids.squeeze(), enc.attention_mask.squeeze(), labels
        else:
            return enc.input_ids.squeeze(), enc.attention_mask.squeeze()


In [20]:
# class EmotionClassifier(nn.Module):
#     def __init__(self, model_name="roberta-base", dropout=0.3):
#         super().__init__()
#         self.encoder = AutoModel.from_pretrained(model_name)
#         self.dropout = nn.Dropout(dropout)
#         self.fc = nn.Linear(self.encoder.config.hidden_size, 5)
    
#     def forward(self, input_ids, attention_mask):
#         out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
#         pooled = out.last_hidden_state[:,0]  # CLS token
#         x = self.dropout(pooled)
#         return self.fc(x)


# tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# train_ds = EmotionDataset(X_train, tokenizer, is_train=True)
# val_ds   = EmotionDataset(X_val, tokenizer, is_train=True)
# test_ds  = EmotionDataset(test, tokenizer, is_train=False)

# train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
# val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False)
# test_loader  = DataLoader(test_ds, batch_size=16, shuffle=False)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [23]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = EmotionClassifier().to(device)
# criterion = nn.BCEWithLogitsLoss()

# optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)   # ⭐ changed to wandb config
# epochs = config.epochs

# total_steps = len(train_loader) * epochs

# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=int(0.1 * total_steps),
#     num_training_steps=total_steps
# )

# def train_model():
#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0

#         for batch in tqdm(train_loader):
#             input_ids, attention_mask, labels = [b.to(device) for b in batch]
            
#             optimizer.zero_grad()
#             outputs = model(input_ids, attention_mask)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             scheduler.step()

#             total_loss += loss.item()
#             wandb.log({"batch_loss": loss.item()}) 

#         epoch_loss = total_loss / len(train_loader)
#         print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}")

#         wandb.log({"train_epoch_loss": epoch_loss})
        
#         validate(epoch)

# def validate(epoch):
#     model.eval()
#     preds_list = []
#     true_list = []

#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids, attention_mask, labels = [b.to(device) for b in batch]
#             outputs = model(input_ids, attention_mask)
#             preds = torch.sigmoid(outputs).cpu().numpy()
#             preds_list.append(preds)
#             true_list.append(labels.cpu().numpy())

#     preds_list = np.vstack(preds_list)
#     true_list = np.vstack(true_list)

#     # Macro F1 calculation across 5 labels
#     from sklearn.metrics import f1_score
#     binary_preds = (preds_list > 0.5).astype(int)
#     f1 = f1_score(true_list, binary_preds, average="macro")
#     print("Validation Macro F1:", f1)
#     wandb.log({"val_macro_f1": f1}) 


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# train_model()

100%|██████████| 384/384 [02:14<00:00,  2.86it/s]


Epoch 1/3 - Loss: 0.4598
Validation Macro F1: 0.7568219698520873


100%|██████████| 384/384 [02:19<00:00,  2.74it/s]


Epoch 2/3 - Loss: 0.2780
Validation Macro F1: 0.7852946232423269


100%|██████████| 384/384 [02:21<00:00,  2.71it/s]


Epoch 3/3 - Loss: 0.1998
Validation Macro F1: 0.8228246844548435


In [25]:
# wandb.finish() 

batch_loss,▇█▆▄▇▅▃▂▄▂▄▆▆▅▅▄▆▄▄▄▅▅▃▄▃▃▃▂▃▂▁▁▁▃▂▂▁▁▂▂
train_epoch_loss,█▃▁
val_macro_f1,▁▄█
batch_loss,0.16326
train_epoch_loss,0.19982
val_macro_f1,0.82282


In [21]:
# save_dir = "roberta_emotion_model"
# os.makedirs(save_dir, exist_ok=True)

# model_path = os.path.join(save_dir, "model.pth")
# torch.save(model.state_dict(), model_path)
# print("Saved:", model_path)

# user = "somya2611"
# upload_handle = f"{user}/roberta-emotion/pyTorch/v1"

# kagglehub.model_upload(upload_handle, "roberta_emotion_model")

# print("Model uploaded to:", upload_handle)


Saved: roberta_emotion_model/model.pth
Uploading Model https://www.kaggle.com/models/somya2611/roberta-emotion/pyTorch/v1 ...
Model 'roberta-emotion' does not exist or access is forbidden for user 'somya2611'. Creating or handling Model...
Model 'roberta-emotion' Created.
Starting upload for file roberta_emotion_model/model.pth


Uploading: 100%|██████████| 499M/499M [00:04<00:00, 118MB/s]  

Upload successful: roberta_emotion_model/model.pth (476MB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/somya2611/roberta-emotion/pyTorch/v1
Model uploaded to: somya2611/roberta-emotion/pyTorch/v1


In [22]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# user = "somya2611"
# download_handle = f"{user}/roberta-emotion/pyTorch/v1"

# model_folder = kagglehub.model_download(download_handle)
# print("Downloaded model folder:", model_folder)


Downloaded model folder: /kaggle/input/roberta-emotion/pytorch/v1/1


In [23]:
# inference_model = EmotionClassifier().to(device)
# state_dict_path = f"{model_folder}/model.pth"

# inference_model.load_state_dict(torch.load(state_dict_path, map_location=device))
# inference_model.eval()

# print("Model loaded for inference!")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded for inference!


In [24]:
# tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# test_ds  = EmotionDataset(test, tokenizer, is_train=False)
# test_loader  = DataLoader(test_ds, batch_size=16, shuffle=False)


# final_preds = []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids, attention_mask = [b.to(device) for b in batch]
#         outputs = inference_model(input_ids, attention_mask)
#         probs = torch.sigmoid(outputs).cpu().numpy()
#         preds = (probs > 0.5).astype(int)
#         final_preds.append(preds)

# final_preds = np.vstack(final_preds)



# submission = pd.DataFrame({
#     "id": test["id"],
#     "anger": final_preds[:, 0],
#     "fear": final_preds[:, 1],
#     "joy": final_preds[:, 2],
#     "sadness": final_preds[:, 3],
#     "surprise": final_preds[:, 4],
# })

# submission.to_csv("submission.csv", index=False)
# submission.head()


,id,anger,fear,joy,sadness,surprise
0,0,1,1,0,0,0
1,1,0,0,0,0,0
2,2,1,1,0,0,1
3,3,0,1,0,0,0
4,4,0,1,0,0,1


# **MODEL FROM SCRATCH**

In [28]:
# import pandas as pd
# import numpy as np
# import re
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import f1_score
# from collections import Counter
# from tqdm.auto import tqdm
# import warnings
# warnings.filterwarnings('ignore')


In [29]:
# class Config:
#     # Model architecture
#     VOCAB_SIZE = 30000          # Increased vocabulary
#     EMBEDDING_DIM = 400         # Larger embeddings
#     HIDDEN_DIM = 512            # Larger hidden size
#     NUM_LAYERS = 3              # More layers
#     NUM_LABELS = 5
#     DROPOUT = 0.4               # Adjusted dropout
#     BIDIRECTIONAL = True
    
#     # Training settings
#     BATCH_SIZE = 32             # Smaller batch for better gradients
#     EPOCHS = 20                 # More epochs
#     LEARNING_RATE = 0.0015      # Adjusted learning rate
#     MAX_LENGTH = 150            # Longer sequences
#     WEIGHT_DECAY = 1e-5         # L2 regularization
    
#     # Augmentation
#     USE_AUGMENTATION = True
#     AUG_PROBABILITY = 0.3
    
#     # Other settings
#     SEED = 42
#     DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     EMOTION_COLS = ['anger', 'fear', 'joy', 'sadness', 'surprise']

# def set_seed(seed=42):
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     if torch.cuda.is_available():
#         torch.cuda.manual_seed_all(seed)

# set_seed(Config.SEED)
# print(f"Using device: {Config.DEVICE}")

Using device: cuda


In [30]:
# wandb.login(key = "1cae1eb0b3009c258573b649b577124df891befe" , relogin=True) 

# wandb.init(
#     project="23f1001420-t32025",
#     name="scratch_emotion_classifier",
#     config={
#         "vocab_size": Config.VOCAB_SIZE,
#         "embedding_dim": Config.EMBEDDING_DIM,
#         "hidden_dim": Config.HIDDEN_DIM,
#         "num_layers": Config.NUM_LAYERS,
#         "num_labels": Config.NUM_LABELS,
#         "dropout": Config.DROPOUT,
#         "bidirectional": Config.BIDIRECTIONAL,
        
#         "batch_size": Config.BATCH_SIZE,
#         "epochs": Config.EPOCHS,
#         "learning_rate": Config.LEARNING_RATE,
#         "max_length": Config.MAX_LENGTH,
#         "weight_decay": Config.WEIGHT_DECAY,
        
#         "augmentation": Config.USE_AUGMENTATION,
#         "aug_prob": Config.AUG_PROBABILITY,
        
#         "seed": Config.SEED,
#     }
# )

# config = wandb.config


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [31]:
# # ============================================================================
# # ENHANCED TEXT PREPROCESSING WITH AUGMENTATION
# # ============================================================================

# class EnhancedTextPreprocessor:
#     def __init__(self):
#         self.word2idx = {'<PAD>': 0, '<UNK>': 1}
#         self.idx2word = {0: '<PAD>', 1: '<UNK>'}
#         self.vocab_size = 2
        
#         # Emotion-specific keywords for better understanding
#         self.emotion_keywords = {
#             'anger': ['angry', 'mad', 'furious', 'annoyed', 'irritated', 'rage'],
#             'fear': ['scared', 'afraid', 'terrified', 'anxious', 'worried', 'nervous'],
#             'joy': ['happy', 'joyful', 'excited', 'glad', 'delighted', 'cheerful'],
#             'sadness': ['sad', 'depressed', 'unhappy', 'miserable', 'heartbroken', 'crying'],
#             'surprise': ['surprised', 'shocked', 'amazed', 'astonished', 'unexpected']
#         }
    
#     def clean_text(self, text):
#         """Enhanced text cleaning"""
#         text = str(text).lower()
        
#         # Preserve important punctuation
#         text = re.sub(r'!+', ' ! ', text)
#         text = re.sub(r'\?+', ' ? ', text)
        
#         # Remove URLs
#         text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        
#         # Handle negations
#         text = re.sub(r"n't", " not", text)
#         text = re.sub(r"'m", " am", text)
#         text = re.sub(r"'re", " are", text)
#         text = re.sub(r"'ve", " have", text)
        
#         # Remove mentions but keep hashtag content
#         text = re.sub(r'@\w+', '', text)
#         text = re.sub(r'#', '', text)
        
#         # Keep letters, numbers, and some punctuation
#         text = re.sub(r'[^a-z0-9\s.,!?\'-]', '', text)
        
#         # Handle repeated characters (e.g., 'soooo' -> 'so')
#         text = re.sub(r'(.)\1{2,}', r'\1\1', text)
        
#         # Remove extra whitespace
#         text = re.sub(r'\s+', ' ', text).strip()
        
#         return text
    
#     def tokenize(self, text):
#         """Enhanced tokenization"""
#         return text.split()
    
#     def augment_text(self, text):
#         """Simple augmentation: synonym replacement"""
#         words = text.split()
#         if len(words) < 3 or np.random.random() > Config.AUG_PROBABILITY:
#             return text
        
#         # Simple word dropout
#         if np.random.random() < 0.3:
#             idx = np.random.randint(0, len(words))
#             words.pop(idx)
        
#         return ' '.join(words)
    
#     def build_vocab(self, texts, max_vocab_size=30000):
#         """Build vocabulary from texts"""
#         print("Building vocabulary...")
#         word_freq = Counter()
        
#         for text in tqdm(texts):
#             cleaned = self.clean_text(text)
#             tokens = self.tokenize(cleaned)
#             word_freq.update(tokens)
        
#         # Add most common words to vocabulary
#         most_common = word_freq.most_common(max_vocab_size - 2)
        
#         for word, freq in most_common:
#             if word not in self.word2idx:
#                 self.word2idx[word] = self.vocab_size
#                 self.idx2word[self.vocab_size] = word
#                 self.vocab_size += 1
        
#         print(f"Vocabulary size: {self.vocab_size}")
#         return self.word2idx
    
#     def text_to_sequence(self, text, max_length=150):
#         """Convert text to sequence of indices"""
#         cleaned = self.clean_text(text)
#         tokens = self.tokenize(cleaned)
        
#         # Convert to indices
#         sequence = [self.word2idx.get(token, 1) for token in tokens]
        
#         # Pad or truncate
#         if len(sequence) < max_length:
#             sequence = sequence + [0] * (max_length - len(sequence))
#         else:
#             sequence = sequence[:max_length]
        
#         return sequence
    
#     def texts_to_sequences(self, texts, max_length=150):
#         """Convert multiple texts to sequences"""
#         sequences = []
#         for text in tqdm(texts, desc="Converting texts"):
#             sequences.append(self.text_to_sequence(text, max_length))
#         return np.array(sequences)

# # ============================================================================
# # DATASET CLASS WITH AUGMENTATION
# # ============================================================================

# class EmotionDataset(Dataset):
#     def __init__(self, sequences, labels, augment=False):
#         self.sequences = torch.LongTensor(sequences)
#         self.labels = torch.FloatTensor(labels)
#         self.augment = augment
    
#     def __len__(self):
#         return len(self.sequences)
    
#     def __getitem__(self, idx):
#         sequence = self.sequences[idx]
        
#         # Simple sequence augmentation: random dropout
#         if self.augment and np.random.random() < 0.2:
#             mask = torch.rand(sequence.shape) > 0.1
#             sequence = sequence * mask.long()
        
#         return {
#             'sequence': sequence,
#             'labels': self.labels[idx]
#         }

# # ============================================================================
# # ENHANCED ATTENTION MECHANISM
# # ============================================================================

# class MultiHeadAttention(nn.Module):
#     def __init__(self, hidden_dim, num_heads=4):
#         super(MultiHeadAttention, self).__init__()
#         self.num_heads = num_heads
#         self.hidden_dim = hidden_dim
#         self.head_dim = hidden_dim // num_heads
        
#         self.query = nn.Linear(hidden_dim, hidden_dim)
#         self.key = nn.Linear(hidden_dim, hidden_dim)
#         self.value = nn.Linear(hidden_dim, hidden_dim)
#         self.out = nn.Linear(hidden_dim, hidden_dim)
        
#     def forward(self, x):
#         batch_size, seq_len, hidden_dim = x.size()
        
#         # Linear projections
#         Q = self.query(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
#         K = self.key(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
#         V = self.value(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
#         # Attention scores
#         scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)
#         attention_weights = F.softmax(scores, dim=-1)
        
#         # Apply attention
#         attended = torch.matmul(attention_weights, V)
#         attended = attended.transpose(1, 2).contiguous().view(batch_size, seq_len, hidden_dim)
        
#         output = self.out(attended)
#         return output

# # ============================================================================
# # ENHANCED MODEL ARCHITECTURE
# # ============================================================================

# class EnhancedEmotionClassifier(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, 
#                  num_labels, dropout=0.4, bidirectional=True):
#         super(EnhancedEmotionClassifier, self).__init__()
        
#         self.hidden_dim = hidden_dim
#         self.num_layers = num_layers
#         self.bidirectional = bidirectional
        
#         # Embedding layer with dropout
#         self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
#         self.embedding_dropout = nn.Dropout(0.2)
        
#         # GRU layers (often better than LSTM for text)
#         self.gru = nn.GRU(
#             embedding_dim,
#             hidden_dim,
#             num_layers,
#             batch_first=True,
#             dropout=dropout if num_layers > 1 else 0,
#             bidirectional=bidirectional
#         )
        
#         # Multi-head attention
#         gru_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
#         self.attention = MultiHeadAttention(gru_output_dim, num_heads=4)
        
#         # Global max pooling
#         self.global_max_pool = nn.AdaptiveMaxPool1d(1)
        
#         # Feature combination
#         combined_dim = gru_output_dim * 2  # attention + max pool
        
#         # Enhanced fully connected layers
#         self.fc1 = nn.Linear(combined_dim, hidden_dim)
#         self.bn1 = nn.BatchNorm1d(hidden_dim)
#         self.dropout1 = nn.Dropout(dropout)
        
#         self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
#         self.bn2 = nn.BatchNorm1d(hidden_dim // 2)
#         self.dropout2 = nn.Dropout(dropout)
        
#         self.fc3 = nn.Linear(hidden_dim // 2, hidden_dim // 4)
#         self.bn3 = nn.BatchNorm1d(hidden_dim // 4)
#         self.dropout3 = nn.Dropout(dropout)
        
#         self.fc_out = nn.Linear(hidden_dim // 4, num_labels)
        
#     def forward(self, x):
#         # Embedding
#         embedded = self.embedding(x)
#         embedded = self.embedding_dropout(embedded)
        
#         # GRU
#         gru_out, _ = self.gru(embedded)
        
#         # Multi-head attention
#         attended = self.attention(gru_out)
        
#         # Mean of attended output
#         attended_mean = torch.mean(attended, dim=1)
        
#         # Max pooling over sequence
#         max_pooled = self.global_max_pool(gru_out.transpose(1, 2)).squeeze(2)
        
#         # Combine features
#         combined = torch.cat([attended_mean, max_pooled], dim=1)
        
#         # Fully connected layers
#         x = self.fc1(combined)
#         x = self.bn1(x)
#         x = F.relu(x)
#         x = self.dropout1(x)
        
#         x = self.fc2(x)
#         x = self.bn2(x)
#         x = F.relu(x)
#         x = self.dropout2(x)
        
#         x = self.fc3(x)
#         x = self.bn3(x)
#         x = F.relu(x)
#         x = self.dropout3(x)
        
#         logits = self.fc_out(x)
        
#         return logits

# # ============================================================================
# # FOCAL LOSS FOR IMBALANCED DATA
# # ============================================================================

# class FocalLoss(nn.Module):
#     def __init__(self, alpha=0.25, gamma=2.0):
#         super(FocalLoss, self).__init__()
#         self.alpha = alpha
#         self.gamma = gamma
        
#     def forward(self, inputs, targets):
#         BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
#         pt = torch.exp(-BCE_loss)
#         F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
#         return F_loss.mean()

# # ============================================================================
# # TRAINING & EVALUATION FUNCTIONS
# # ============================================================================

# def train_epoch(model, data_loader, optimizer, criterion, device):
#     model.train()
#     losses = []
    
#     progress_bar = tqdm(data_loader, desc='Training')
    
#     for batch in progress_bar:
#         sequences = batch['sequence'].to(device)
#         labels = batch['labels'].to(device)
        
#         optimizer.zero_grad()
        
#         logits = model(sequences)
#         loss = criterion(logits, labels)
        
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#         optimizer.step()
        
#         losses.append(loss.item())
#         progress_bar.set_postfix({'loss': np.mean(losses)})
    
#     return np.mean(losses)

# def eval_model(model, data_loader, criterion, device, threshold=0.5):
#     model.eval()
#     losses = []
#     all_predictions = []
#     all_labels = []
    
#     with torch.no_grad():
#         for batch in tqdm(data_loader, desc='Evaluating'):
#             sequences = batch['sequence'].to(device)
#             labels = batch['labels'].to(device)
            
#             logits = model(sequences)
#             loss = criterion(logits, labels)
            
#             losses.append(loss.item())
            
#             probs = torch.sigmoid(logits)
#             preds = (probs > threshold).int()
            
#             all_predictions.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
    
#     all_predictions = np.array(all_predictions)
#     all_labels = np.array(all_labels)
    
#     f1_macro = f1_score(all_labels, all_predictions, average='macro', zero_division=0)
#     f1_per_class = f1_score(all_labels, all_predictions, average=None, zero_division=0)
    
#     return np.mean(losses), f1_macro, f1_per_class

# def predict(model, data_loader, device, threshold=0.5):
#     model.eval()
#     all_predictions = []
    
#     with torch.no_grad():
#         for batch in tqdm(data_loader, desc='Predicting'):
#             sequences = batch['sequence'].to(device)
            
#             logits = model(sequences)
#             probs = torch.sigmoid(logits)
#             preds = (probs > threshold).int()
            
#             all_predictions.extend(preds.cpu().numpy())
    
#     return np.array(all_predictions)

In [32]:
# # ============================================================================
# # LOAD AND PREPARE DATA
# # ============================================================================

# print("Loading data...")
# train_df = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/train.csv')

# print(f"Train shape: {train_df.shape}")
# print(f"\nFirst few rows:")
# print(train_df.head())

# print("\nEmotion distribution:")
# for col in Config.EMOTION_COLS:
#     print(f"{col}: {train_df[col].sum()} ({train_df[col].mean()*100:.2f}%)")

# # Initialize preprocessor
# preprocessor = EnhancedTextPreprocessor()

# # Build vocabulary
# preprocessor.build_vocab(train_df['text'].tolist(), max_vocab_size=Config.VOCAB_SIZE)

# # Convert texts to sequences
# print("\nConverting texts to sequences...")
# sequences = preprocessor.texts_to_sequences(
#     train_df['text'].tolist(), 
#     max_length=Config.MAX_LENGTH
# )

# labels = train_df[Config.EMOTION_COLS].values

# # Split data with stratification
# train_seq, val_seq, train_labels, val_labels = train_test_split(
#     sequences, labels, test_size=0.15, random_state=Config.SEED
# )

# print(f"\nTrain size: {len(train_seq)}")
# print(f"Validation size: {len(val_seq)}")

# # Create datasets
# train_dataset = EmotionDataset(train_seq, train_labels, augment=Config.USE_AUGMENTATION)
# val_dataset = EmotionDataset(val_seq, val_labels, augment=False)

# train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

# # ============================================================================
# # INITIALIZE MODEL
# # ============================================================================

# print("\nInitializing enhanced model...")
# model = EnhancedEmotionClassifier(
#     vocab_size=preprocessor.vocab_size,
#     embedding_dim=Config.EMBEDDING_DIM,
#     hidden_dim=Config.HIDDEN_DIM,
#     num_layers=Config.NUM_LAYERS,
#     num_labels=Config.NUM_LABELS,
#     dropout=Config.DROPOUT,
#     bidirectional=Config.BIDIRECTIONAL
# )
# model.to(Config.DEVICE)

# total_params = sum(p.numel() for p in model.parameters())
# trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(f"Total parameters: {total_params:,}")
# print(f"Trainable parameters: {trainable_params:,}")

# # Use Focal Loss for better handling of class imbalance
# criterion = FocalLoss(alpha=0.25, gamma=2.0)
# optimizer = torch.optim.AdamW(
#     model.parameters(), 
#     lr=Config.LEARNING_RATE,
#     weight_decay=Config.WEIGHT_DECAY
# )
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
#     optimizer, T_0=5, T_mult=2
# )

Loading data...
Train shape: (6827, 8)

First few rows:
   id                                               text  anger  fear  joy  \
0   0  the dentist that did the work apparently did a...      1     0    0   
1   1  i'm gonna absolutely ~~suck~~ be terrible duri...      0     1    0   
2   2  bridge: so leave me drowning calling houston, ...      0     1    0   
3   3  after that mess i went to see my now ex-girlfr...      1     1    0   
4   4  as he stumbled i ran off, afraid it might some...      0     1    0   

   sadness  surprise                    emotions  
0        1         0         ['anger' 'sadness']  
1        1         0          ['fear' 'sadness']  
2        1         0          ['fear' 'sadness']  
3        1         0  ['anger' 'fear' 'sadness']  
4        0         0                    ['fear']  

Emotion distribution:
anger: 808 (11.84%)
fear: 3860 (56.54%)
joy: 1660 (24.32%)
sadness: 2171 (31.80%)
surprise: 1999 (29.28%)
Building vocabulary...


  0%|          | 0/6827 [00:00<?, ?it/s]

Vocabulary size: 11092

Converting texts to sequences...


Converting texts:   0%|          | 0/6827 [00:00<?, ?it/s]


Train size: 5802
Validation size: 1025

Initializing enhanced model...
Total parameters: 22,108,229
Trainable parameters: 22,108,229


In [33]:
# # ============================================================================
# # TRAINING LOOP
# # ============================================================================

# print("\n" + "="*60)
# print("STARTING TRAINING")
# print("="*60)

# best_f1 = 0
# patience_counter = 0
# patience = 5

# history = {'train_loss': [], 'val_loss': [], 'val_f1': []}

# for epoch in range(Config.EPOCHS):
#     print(f'\nEpoch {epoch + 1}/{Config.EPOCHS}')
#     print('-' * 60)
    
#     train_loss = train_epoch(model, train_loader, optimizer, criterion, Config.DEVICE)
#     val_loss, val_f1, val_f1_per_class = eval_model(model, val_loader, criterion, Config.DEVICE)
    
#     scheduler.step()
    
#     history['train_loss'].append(train_loss)
#     history['val_loss'].append(val_loss)
#     history['val_f1'].append(val_f1)

#     wandb.log({
#         "train_loss": train_loss,
#         "val_loss": val_loss,
#         "val_f1": val_f1,
#         "learning_rate": optimizer.param_groups[0]['lr']
#     })
    
#     print(f'\nTrain Loss: {train_loss:.4f}')
#     print(f'Val Loss: {val_loss:.4f}')
#     print(f'Val Macro F1: {val_f1:.4f}')
#     print(f'\nPer-class F1 scores:')
#     for emotion, f1 in zip(Config.EMOTION_COLS, val_f1_per_class):
#         wandb.log({f"F1_{emotion}": f1})
#         print(f'  {emotion}: {f1:.4f}')
    
#     # Save best model
#     if val_f1 > best_f1:
#         best_f1 = val_f1
#         patience_counter = 0
#         torch.save({
#             'model_state_dict': model.state_dict(),
#             'preprocessor': preprocessor,
#             'config': Config
#         }, 'best_model.pt')
#         print(f'\n✓ New best model saved! (F1: {best_f1:.4f})')
#     else:
#         patience_counter += 1
#         if patience_counter >= patience:
#             print(f"\nEarly stopping triggered after {epoch+1} epochs")
#             break

# print("\n" + "="*60)
# print(f"TRAINING COMPLETED - Best F1: {best_f1:.4f}")
# print("="*60)



STARTING TRAINING

Epoch 1/20
------------------------------------------------------------


Training:   0%|          | 0/182 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/33 [00:00<?, ?it/s]


Train Loss: 0.0401
Val Loss: 0.0362
Val Macro F1: 0.1632

Per-class F1 scores:
  anger: 0.0000
  fear: 0.6723
  joy: 0.0000
  sadness: 0.1439
  surprise: 0.0000

✓ New best model saved! (F1: 0.1632)

Epoch 2/20
------------------------------------------------------------


Training:   0%|          | 0/182 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/33 [00:00<?, ?it/s]


Train Loss: 0.0206
Val Loss: 0.0298
Val Macro F1: 0.5853

Per-class F1 scores:
  anger: 0.3152
  fear: 0.7502
  joy: 0.6337
  sadness: 0.4903
  surprise: 0.7370

✓ New best model saved! (F1: 0.5853)

Epoch 20/20
------------------------------------------------------------


Training:   0%|          | 0/182 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/33 [00:00<?, ?it/s]


Train Loss: 0.0192
Val Loss: 0.0295
Val Macro F1: 0.6175

Per-class F1 scores:
  anger: 0.4444
  fear: 0.7745
  joy: 0.6256
  sadness: 0.5574
  surprise: 0.6854

✓ New best model saved! (F1: 0.6175)

TRAINING COMPLETED - Best F1: 0.6175


In [34]:
# wandb.finish()

F1_anger,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▆█
F1_fear,▃▅▅▅▅▅▅▄▃▃▄▆▅▆▆▁▅▇▇█
F1_joy,▁▁▁▁▁▁▁▁▁▃▆▇▇▇▇▇████
F1_sadness,▃▁▁▁▁▄▄▂▆▇▆▇▇▆▇▅▇█▇█
F1_surprise,▁▁▂▆▆▇▅▇▇▇█▇███▇▇███
learning_rate,▇▆▃▂██▇▇▆▄▃▂▂▁████▇▇
train_loss,█▇▇▆▆▆▆▆▅▄▄▃▂▂▂▃▃▂▁▁
val_f1,▁▁▁▃▃▄▃▃▄▅▆▆▆▆▆▅▆▇██
val_loss,███▇▇▇▇▆▄▄▃▂▂▂▂▇▂▂▁▁
F1_anger,0.44444
F1_fear,0.77447


In [ ]:
# import kagglehub
# # torch.save(model.state_dict(), "emotion_classifier.pth")
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'preprocessor': preprocessor,
#     'config': Config
# }, 'best_model.pt')


# # Upload to KaggleHub
# user = "somya2611" 
# emotion_handle = f"{user}/emotion-classifier/pyTorch/v1"
# os.makedirs("emotion_model_dir", exist_ok=True)
# os.rename("best_model.pt", "emotion_model_dir/best_model.pt")
# kagglehub.model_upload(emotion_handle, "emotion_model_dir")

In [ ]:
# import kagglehub
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
# user = "somya2611" 
# emotion_handle = f"{user}/emotion-classifier/pyTorch/v1"

# model_path = kagglehub.model_download(emotion_handle)

# checkpoint = torch.load(f"{model_path}/best_model.pt", map_location=device, weights_only=False)



# # Recreate model architecture
# model = EnhancedEmotionClassifier(
#     vocab_size=checkpoint['preprocessor'].vocab_size,
#     embedding_dim=checkpoint['config'].EMBEDDING_DIM,
#     hidden_dim=checkpoint['config'].HIDDEN_DIM,
#     num_layers=checkpoint['config'].NUM_LAYERS,
#     num_labels=checkpoint['config'].NUM_LABELS,
#     dropout=checkpoint['config'].DROPOUT,
#     bidirectional=checkpoint['config'].BIDIRECTIONAL
# )

# # Load trained weights
# model.load_state_dict(checkpoint['model_state_dict'])
# model.eval()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# # Get preprocessor
# preprocessor = checkpoint['preprocessor']

# # Load test data
# test_df = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/test.csv')

# # Process and predict
# test_sequences = preprocessor.texts_to_sequences(
#     test_df['text'].tolist(),
#     max_length=checkpoint['config'].MAX_LENGTH
# )

# # Create dataset and loader
# test_labels_dummy = torch.zeros((len(test_sequences), 5))
# test_dataset = EmotionDataset(test_sequences, test_labels_dummy, augment=False)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

# # Predict
# predictions = predict(model, test_loader, device, threshold=0.5)

# # Create submission
# submission = pd.DataFrame(predictions, columns=['anger', 'fear', 'joy', 'sadness', 'surprise'])
# if 'id' in test_df.columns:
#     submission.insert(0, 'id', test_df['id'])
# submission.to_csv('submission.csv', index=False)
# print("Predictions saved!")

# print(submission.head())